<a href="https://colab.research.google.com/github/Manojksen/8-Week-Sql-Challenge-Danny-ma/blob/main/automate_every_5min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance --upgrade
import yfinance as yf
import pandas as pd

In [ ]:
df = pd.read_csv("/content/FNO Stocks - All FO Stocks List, Technical Analysis Scanner.csv")
tickers = df['Symbol'].tolist()
tickers = [ticker + '.NS' for ticker in tickers]
all_tickers=tickers


In [ ]:
all_tickers= ["AARTIIND.NS","ABBOTINDIA.NS","ABFRL.NS",
    'ADANIENT.NS','ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AMBUJACEM.NS', 'ABB.NS',
    'HDFCLIFE.NS', 'ANGELONE.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AUBANK.NS', 'AUROPHARMA.NS',
    'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BANDHANBNK.NS',
    'BANKBARODA.NS', 'BEL.NS', 'BERGEPAINT.NS', 'BHARTIARTL.NS', 'BIOCON.NS', 'BOSCHLTD.NS',
    'BPCL.NS', 'BRITANNIA.NS', 'CANBK.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'COALINDIA.NS',
    'COLPAL.NS', 'CONCOR.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'DABUR.NS',
    'DALBHARAT.NS', 'DEEPAKNTR.NS', 'DIVISLAB.NS', 'DLF.NS', 'DRREDDY.NS', 'EICHERMOT.NS',
    'ESCORTS.NS', 'EXIDEIND.NS', 'FEDERALBNK.NS', 'GAIL.NS', 'GODREJCP.NS', 'GODREJPROP.NS',
    'GRASIM.NS', 'HAVELLS.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS',
    'HINDPETRO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDEA.NS',
    'IDFCFIRSTB.NS', 'IGL.NS', 'INDHOTEL.NS', 'INDIGO.NS',"INOXWIND.NS", 'INDUSINDBK.NS', 'INDUSTOWER.NS',
    'INFY.NS', 'IOC.NS', 'IRCTC.NS', 'ITC.NS', 'JINDALSTEL.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS',
    'KOTAKBANK.NS', 'GLENMARK.NS', 'LALPATHLAB.NS', 'LICHSGFIN.NS', 'LT.NS', 'LTIM.NS', 'LTTS.NS',
    'LUPIN.NS', 'M&M.NS', 'MARICO.NS', 'MARUTI.NS', 'MUTHOOTFIN.NS',"MANKIND.NS", 'NAUKRI.NS', 'NAVINFLUOR.NS',"NBCC.NS",
    'NESTLEIND.NS', 'NMDC.NS', 'NTPC.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'PAGEIND.NS', 'PEL.NS',
    'PETRONET.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'POLYCAB.NS', 'POWERGRID.NS', 'RECLTD.NS',
    'RELIANCE.NS', 'SAIL.NS', 'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS', 'SIEMENS.NS', 'SRF.NS',
    'SUNPHARMA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS',
    'TATASTEEL.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TRENT.NS', 'TVSMOTOR.NS',
    'UBL.NS', 'ULTRACEMCO.NS', 'UPL.NS', 'VEDL.NS', 'VOLTAS.NS', 'WIPRO.NS', 'ZEEL.NS', 'ZYDUSLIFE.NS'
]

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time
import threading

# Timezone
ist = pytz.timezone('Asia/Kolkata')


df = pd.read_csv("/content/FNO Stocks - All FO Stocks List, Technical Analysis Scanner.csv")
tickers = df['Symbol'].tolist()
tickers = [ticker + '.NS' for ticker in tickers]
all_tickers=tickers



# ATR Calculation
def compute_atr(df, period=14):
    high_low = df['High'] - df['Low']
    high_close = (df['High'] - df['Close'].shift()).abs()
    low_close = (df['Low'] - df['Close'].shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

# Check if last candle is complete
def is_last_candle_complete(last_timestamp):
    now_ist = datetime.now(ist)
    return (now_ist - last_timestamp) > timedelta(minutes=5)


# Manual stop flag
stop_flag = False

# Manual Stop Handler in background
def listen_for_stop():
    global stop_flag
    input("🛑 Press ENTER to stop the screener anytime...\n")
    stop_flag = True

# Screener function
def run_screener():
    now_ist = datetime.now(ist)
    print(f"\n🕒 Running Screener at {now_ist.strftime('%Y-%m-%d %H:%M:%S')} IST")

    data = yf.download(all_tickers, period="7d", interval="5m", group_by='ticker', threads=True)

    buy_signals = []
    sell_signals = []

    for ticker in all_tickers:
        try:
            df = data[ticker].copy().sort_index()

            if len(df) < 241:
                print(f"{ticker}: Not enough data ({len(df)} rows), skipping")
                continue

            last_timestamp = df.index[-1].to_pydatetime().astimezone(ist)
            if not is_last_candle_complete(last_timestamp):
                df = df.iloc[:-1]

            df['ATR_14'] = compute_atr(df)
            df.dropna(inplace=True)

            if len(df) < 241:
                print(f"{ticker}: Not enough data after ATR, skipping")
                continue

            current_close = df['Close'].iloc[-1]
            current_volume = df['Volume'].iloc[-1]
            current_atr = df['ATR_14'].iloc[-1]

            prev_240 = df.iloc[-241:-1]
            max_close_240 = prev_240['Close'].max()
            min_close_240 = prev_240['Close'].min()
            avg_vol_240 = prev_240['Volume'].mean()

            # Buy logic
            if (
                current_close > max_close_240 and
                current_volume > 2 * avg_vol_240 and
                current_close > 5
            ):
                buy_signals.append({
                    "Ticker": ticker,
                    "Current Close": current_close,
                    "Prev 240 Max Close": max_close_240,
                    "Current Volume": current_volume,
                    "Prev 240 Avg Volume": avg_vol_240,
                    "ATR(14) 5min": current_atr
                })

            # Sell logic
            if (
                current_close < min_close_240 and
                current_volume > 2 * avg_vol_240 and
                current_close > 5
            ):
                sell_signals.append({
                    "Ticker": ticker,
                    "Current Close": current_close,
                    "Prev 240 Min Close": min_close_240,
                    "Current Volume": current_volume,
                    "Prev 240 Avg Volume": avg_vol_240,
                    "ATR(14) 5min": current_atr
                })

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    buy_df = pd.DataFrame(buy_signals)
    sell_df = pd.DataFrame(sell_signals)

    combined_df = pd.DataFrame()
    if not buy_df.empty:
        buy_df['direction'] = 'BUY'
        combined_df = pd.concat([combined_df, buy_df], ignore_index=True)

    if not sell_df.empty:
        sell_df['direction'] = 'SELL'
        combined_df = pd.concat([combined_df, sell_df], ignore_index=True)

    if not combined_df.empty:
        now_ist_str = now_ist.strftime("%Y-%m-%d %H:%M")
        combined_df['entry_time'] = now_ist_str
        combined_df['entry'] = combined_df['Current Close']
        combined_df['sl-m'] = (combined_df['ATR(14) 5min'] * 1.5).round(2)

        result_df = combined_df[['Ticker', 'entry_time', 'direction', 'entry', 'sl-m']]
        print("\n📌 Final Signals:")
        print(result_df)
    else:
        print("🔍 No signals this round.")

# ====================
# 🔁 Aligned 5-min Loop
# ====================
threading.Thread(target=listen_for_stop, daemon=True).start()

try:
    while not stop_flag:
        run_screener()

        # Wait till next 5-minute multiple
        now = datetime.now(ist)
        seconds_to_next_5min = 300 - ((now.minute * 60 + now.second) % 300)
        print(f"⏳ Waiting {seconds_to_next_5min} seconds until next 5-min mark...\n")
        time.sleep(seconds_to_next_5min)

except KeyboardInterrupt:
    print("🛑 Screener manually stopped.")




🕒 Running Screener at 2025-08-13 09:12:49 IST


/tmp/ipython-input-2225652310.py:48: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers, period="7d", interval="5m", group_by='ticker', threads=True)
[**********************69%********               ]  154 of 222 completedERROR:yfinance:HTTP Error 404: 
[**********************72%**********             ]  159 of 222 completedERROR:yfinance:HTTP Error 404: 
[*********************100%***********************]  222 of 222 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BANKNIFTY.NS', 'NIFTY.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=7d) (Yahoo error = "No data found, symbol may be delisted")')


BANKNIFTY.NS: Not enough data after ATR, skipping
NIFTY.NS: Not enough data after ATR, skipping

📌 Final Signals:
          Ticker        entry_time direction        entry   sl-m
0     HDFCAMC.NS  2025-08-13 09:12      SELL  5454.000000  13.34
1  MUTHOOTFIN.NS  2025-08-13 09:12      SELL  2514.899902   8.38
2        RVNL.NS  2025-08-13 09:12      SELL   323.450012   4.75
⏳ Waiting 100 seconds until next 5-min mark...

🛑 Screener manually stopped.


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time
import threading

# Timezone
ist = pytz.timezone('Asia/Kolkata')


df = pd.read_csv("/content/FNO Stocks - All FO Stocks List, Technical Analysis Scanner.csv")
tickers = df['Symbol'].tolist()
tickers = [ticker + '.NS' for ticker in tickers]
all_tickers=tickers

# Prepare stock notes DataFrame from same CSV
notes_df = df[['Symbol', 'weekly', 'my thought']].copy()
notes_df['Symbol'] = notes_df['Symbol'].str.strip().str.upper()


# ATR Calculation
def compute_atr(df, period=14):
    high_low = df['High'] - df['Low']
    high_close = (df['High'] - df['Close'].shift()).abs()
    low_close = (df['Low'] - df['Close'].shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

# Check if last candle is complete
def is_last_candle_complete(last_timestamp):
    now_ist = datetime.now(ist)
    return (now_ist - last_timestamp) > timedelta(minutes=5)


# Manual stop flag
stop_flag = False

# Manual Stop Handler in background
def listen_for_stop():
    global stop_flag
    input("🛑 Press ENTER to stop the screener anytime...\n")
    stop_flag = True

# Screener function
def run_screener():
    now_ist = datetime.now(ist)
    print(f"\n🕒 Running Screener at {now_ist.strftime('%Y-%m-%d %H:%M:%S')} IST")

    data = yf.download(all_tickers, period="7d", interval="5m", group_by='ticker', threads=True)

    buy_signals = []
    sell_signals = []

    for ticker in all_tickers:
        try:
            df = data[ticker].copy().sort_index()

            if len(df) < 241:
                print(f"{ticker}: Not enough data ({len(df)} rows), skipping")
                continue

            last_timestamp = df.index[-1].to_pydatetime().astimezone(ist)
            if not is_last_candle_complete(last_timestamp):
                df = df.iloc[:-1]

            df['ATR_14'] = compute_atr(df)
            df.dropna(inplace=True)

            if len(df) < 241:
                print(f"{ticker}: Not enough data after ATR, skipping")
                continue

            current_close = df['Close'].iloc[-1]
            current_volume = df['Volume'].iloc[-1]
            current_atr = df['ATR_14'].iloc[-1]

            prev_240 = df.iloc[-241:-1]
            max_close_240 = prev_240['Close'].max()
            min_close_240 = prev_240['Close'].min()
            avg_vol_240 = prev_240['Volume'].mean()

            # Buy logic
            if (
                current_close > max_close_240 and
                current_volume > 2 * avg_vol_240 and
                current_close > 5
            ):
                buy_signals.append({
                    "Ticker": ticker,
                    "Current Close": current_close,
                    "Prev 240 Max Close": max_close_240,
                    "Current Volume": current_volume,
                    "Prev 240 Avg Volume": avg_vol_240,
                    "ATR(14) 5min": current_atr
                })

            # Sell logic
            if (
                current_close < min_close_240 and
                current_volume > 2 * avg_vol_240 and
                current_close > 5
            ):
                sell_signals.append({
                    "Ticker": ticker,
                    "Current Close": current_close,
                    "Prev 240 Min Close": min_close_240,
                    "Current Volume": current_volume,
                    "Prev 240 Avg Volume": avg_vol_240,
                    "ATR(14) 5min": current_atr
                })

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    buy_df = pd.DataFrame(buy_signals)
    sell_df = pd.DataFrame(sell_signals)

    combined_df = pd.DataFrame()
    if not buy_df.empty:
        buy_df['direction'] = 'BUY'
        combined_df = pd.concat([combined_df, buy_df], ignore_index=True)

    if not sell_df.empty:
        sell_df['direction'] = 'SELL'
        combined_df = pd.concat([combined_df, sell_df], ignore_index=True)

    if not combined_df.empty:
        now_ist_str = now_ist.strftime("%Y-%m-%d %H:%M")
        combined_df['entry_time'] = now_ist_str
        combined_df['entry'] = combined_df['Current Close']
        combined_df['sl-m'] = (combined_df['ATR(14) 5min'] * 1.5).round(2)

        result_df = combined_df[['Ticker', 'entry_time', 'direction', 'entry', 'sl-m']]

        # Extract base symbol (without .NS) for merging
        result_df['Symbol'] = result_df['Ticker'].str.replace('.NS', '', regex=False).str.upper()

        # Merge with notes
        enriched_df = result_df.merge(notes_df, on='Symbol', how='left')

        # Reorder columns
        enriched_df = enriched_df[['Ticker', 'entry_time', 'direction', 'entry', 'sl-m', 'weekly', 'my thought']]

        print("\n📌 Final Signals with Weekly View and My Thought:")
        print(enriched_df)

    else:
        print("🔍 No signals this round.")

# ====================
# 🔁 Aligned 5-min Loop
# ====================
threading.Thread(target=listen_for_stop, daemon=True).start()

try:
    while not stop_flag:
        run_screener()

        # Wait till next 5-minute multiple
        now = datetime.now(ist)
        seconds_to_next_5min = 300 - ((now.minute * 60 + now.second) % 300)
        print(f"⏳ Waiting {seconds_to_next_5min} seconds until next 5-min mark...\n")
        time.sleep(seconds_to_next_5min)

except KeyboardInterrupt:
    print("🛑 Screener manually stopped.")



/tmp/ipython-input-176237620.py:51: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers, period="7d", interval="5m", group_by='ticker', threads=True)



🕒 Running Screener at 2025-08-13 09:21:02 IST


[********************  41%                       ]  90 of 222 completedERROR:yfinance:HTTP Error 404: 
[**********************63%*****                  ]  140 of 222 completedERROR:yfinance:HTTP Error 404: 
[*********************100%***********************]  222 of 222 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['NIFTY.NS', 'BANKNIFTY.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=7d) (Yahoo error = "No data found, symbol may be delisted")')


BANKNIFTY.NS: Not enough data after ATR, skipping
NIFTY.NS: Not enough data after ATR, skipping


/tmp/ipython-input-176237620.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Symbol'] = result_df['Ticker'].str.replace('.NS', '', regex=False).str.upper()


...


🕒 Running Screener at 2025-08-13 16:20:00 IST


[*********************100%***********************]  222 of 222 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['NIFTY.NS', 'BANKNIFTY.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=7d) (Yahoo error = "No data found, symbol may be delisted")')


BANKNIFTY.NS: Not enough data after ATR, skipping
NIFTY.NS: Not enough data after ATR, skipping


/tmp/ipython-input-176237620.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Symbol'] = result_df['Ticker'].str.replace('.NS', '', regex=False).str.upper()



📌 Final Signals with Weekly View and My Thought:
          Ticker        entry_time direction        entry  sl-m   weekly  \
0  LAURUSLABS.NS  2025-08-13 16:20       BUY   865.900024  2.56  Bullish   
1       IRCTC.NS  2025-08-13 16:20       BUY   728.900024  2.00  Neutral   
2    UNITDSPR.NS  2025-08-13 16:20       BUY  1307.500000  2.31      NaN   

     my thought  
0  full bullish  
1         check  
2           NaN  
⏳ Waiting 281 seconds until next 5-min mark...



In [ ]:
data = yf.download(all_tickers, period="7d", interval="5m", group_by='ticker', threads=True)

/tmp/ipython-input-5-4211688728.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers, period="7d", interval="5m", group_by='ticker', threads=True)
[*********************100%***********************]  188 of 188 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MINDTREE.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=7d) (Yahoo error = "No data found, symbol may be delisted")')


In [ ]:
data.tail()

Ticker                    BRITANNIA.NS                                 \
Price                             Open    High     Low   Close Volume   
2025-07-08 08:10:00+00:00       5812.5  5821.5  5812.0  5821.0   3766   
2025-07-08 08:15:00+00:00       5819.0  5821.0  5817.0  5818.5   2030   
2025-07-08 08:20:00+00:00       5818.0  5819.5  5817.5  5819.5   1342   
2025-07-08 08:25:00+00:00       5819.0  5825.0  5817.5  5824.5   2574   
2025-07-08 08:30:00+00:00       5824.5  5824.5  5824.0  5824.5    354   

Ticker                         UPL.NS                                      \
Price                            Open        High         Low       Close   
2025-07-08 08:10:00+00:00  670.049988  670.099976  668.750000  669.400024   
2025-07-08 08:15:00+00:00  669.500000  669.599976  669.099976  669.549988   
2025-07-08 08:20:00+00:00  669.299988  669.750000  669.000000  669.549988   
2025-07-08 08:25:00+00:00  669.549988  670.099976  669.200012  669.849976   
2025-07-08 08:30:00+00:00  669.650024  671.000000  669.650024  670.799988   

Ticker                            ... ULTRACEMCO.NS                    \
Price                     Volume  ...          Open     High      Low   
2025-07-08 08:10:00+00:00   8631  ...       12465.0  12467.0  12460.0   
2025-07-08 08:15:00+00:00   5129  ...       12467.0  12473.0  12459.0   
2025-07-08 08:20:00+00:00   4555  ...       12470.0  12474.0  12464.0   
2025-07-08 08:25:00+00:00   4675  ...       12466.0  12469.0  12461.0   
2025-07-08 08:30:00+00:00   4134  ...       12467.0  12469.0  12462.0   

Ticker                                      COFORGE.NS               \
Price                        Close Volume         Open         High   
2025-07-08 08:10:00+00:00  12466.0    959  1975.000000  1979.900024   
2025-07-08 08:15:00+00:00  12470.0   1257  1979.000000  1979.099976   
2025-07-08 08:20:00+00:00  12466.0   1271  1978.099976  1978.300049   
2025-07-08 08:25:00+00:00  12467.0   1179  1975.500000  1975.800049   
2025-07-08 08:30:00+00:00  12464.0    147  1971.800049  1972.099976   

Ticker                                                      
Price                              Low        Close Volume  
2025-07-08 08:10:00+00:00  1974.599976  1978.500000  14052  
2025-07-08 08:15:00+00:00  1976.599976  1978.199951  10237  
2025-07-08 08:20:00+00:00  1974.000000  1975.500000  10376  
2025-07-08 08:25:00+00:00  1970.000000  1972.000000  19062  
2025-07-08 08:30:00+00:00  1970.000000  1970.400024   6685  

[5 rows x 941 columns]

In [ ]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime

# Load F&O Tickers
df = pd.read_csv("/content/FNO Stocks - All FO Stocks List, Technical Analysis Scanner.csv")
tickers = df['Symbol'].str.strip().str.upper().tolist()
tickers = [ticker + '.NS' for ticker in tickers]

# Output folder
output_folder = "fo_5min_data"
os.makedirs(output_folder, exist_ok=True)

# Download 5-min data for last 60 days
for ticker in tickers:
    try:
        print(f"📥 Downloading {ticker} ...")
        data = yf.download(ticker, period="60d", interval="5m", progress=False)

        if data.empty:
            print(f"⚠️ No data for {ticker}, skipping.")
            continue

        # Save as CSV
        filename = os.path.join(output_folder, f"{ticker.replace('.NS','')}_5min.csv")
        data.to_csv(filename)
        print(f"✅ Saved: {filename}")

    except Exception as e:
        print(f"❌ Error with {ticker}: {e}")


📥 Downloading ATGL.NS ...


/tmp/ipython-input-4-2200874810.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="60d", interval="5m", progress=False)


✅ Saved: fo_5min_data/ATGL_5min.csv
📥 Downloading MGL.NS ...


/tmp/ipython-input-4-2200874810.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="60d", interval="5m", progress=False)


✅ Saved: fo_5min_data/MGL_5min.csv
📥 Downloading HINDPETRO.NS ...


...

✅ Saved: fo_5min_data/ABCAPITAL_5min.csv
📥 Downloading OBEROIRLTY.NS ...


/tmp/ipython-input-4-2200874810.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="60d", interval="5m", progress=False)


✅ Saved: fo_5min_data/OBEROIRLTY_5min.csv
📥 Downloading PHOENIXLTD.NS ...


/tmp/ipython-input-4-2200874810.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="60d", interval="5m", progress=False)


✅ Saved: fo_5min_data/PHOENIXLTD_5min.csv
